# Bien penser à changer le CSV et le nom des DF !!

In [1]:
from selenium import webdriver
from selenium_stealth import stealth
from bs4 import BeautifulSoup
import pandas as pd
import re, time, random

In [3]:
df_data_villes_error_1 = pd.read_csv("villes_error_1.csv", 
                                    delimiter=",",
                                    dtype=str,
                                    index_col=0)


In [4]:
df_data_villes_error_1.head()


,#Code_commune_INSEE,Lib_MA,departement,Code_postal,Nom_de_la_commune_url,url
Unnamed: 0,,,,,,
17928,44009,BASSE GOULAINE,44,44115,basse-goulaine,https://www.meilleursagents.com/prix-immobilie...
17939,44020,BOUGUENAIS,44,44340,bouguenais,https://www.meilleursagents.com/prix-immobilie...
17940,44020,LES COUETS,44,44340,les-couets,https://www.meilleursagents.com/prix-immobilie...
17957,44035,LA CHAPELLE SUR ERDRE,44,44240,la-chapelle-sur-erdre,https://www.meilleursagents.com/prix-immobilie...
17974,44052,DONGES,44,44480,donges,https://www.meilleursagents.com/prix-immobilie...


In [5]:
def extract_number(text):
    """Nettoyer le texte et transformer en float"""
    clean_text = text.replace("\u202f", "").replace("\xa0", "").replace("€", "").strip()
    clean_text = clean_text.replace(",", ".")
    return float(clean_text)

# Selenium setup (Chrome auto-managed by Selenium Manager; Chrome must be installed)
options = webdriver.ChromeOptions()
options.add_argument("--lang=fr-FR")
options.add_argument("--disable-blink-features=AutomationControlled")
# Uncomment to persist cookies/session between runs:
# options.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Local\Google\Chrome\User Data\ScrapeProfile")

driver = webdriver.Chrome(options=options)
stealth(driver,
    languages=["fr-FR", "fr"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)
driver.set_window_size(1366, 768)

# Lancer le navigateur
driver.get("https://www.meilleursagents.com/prix-immobilier/")

In [6]:
# Liste pour stocker toutes les données
all_data = []
error_data = []

for index, row in df_data_villes_error_1.iterrows():
    url = row['url']
    code_insee = row['#Code_commune_INSEE']
    try:
        # Human-like pacing
        time.sleep(random.uniform(1.8, 3.6))

        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        data_dict = {}
        # 1-Ajouter la ville
        title_text = soup.find("title").get_text().strip()
        ville = title_text.split("Prix immobilier")[1].split("(")[0].strip()
        data_dict["ville"] = ville
        
        #2-ajouter prix achat+loyer au m2 appt et maison
        raw_text3 = soup.find_all("ul", class_="prices-summary__price-range")
        for ul in raw_text3:
            li_items = ul.find_all("li")
            label = li_items[0].get_text().strip().lower()
            main_value = extract_number(li_items[1].get_text())
            # Nettoyer le texte pour récupérer min et max correctement
            range_text = li_items[2].get_text()
            range_text = range_text.replace("\u202f", "").replace("\xa0", "").replace(",", ".")
            # Extraire tous les nombres sous forme de float
            numbers_in_range = [float(n) for n in re.findall(r"[\d\.]+", range_text)]
            if "prix" in label:
                if "prix_appartement" not in data_dict:
                    data_dict["prix_appartement"] = main_value
                    data_dict["min_appartement"] = numbers_in_range[0]
                    data_dict["max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["prix_maison"] = main_value
                    data_dict["min_maison"] = numbers_in_range[0]
                    data_dict["max_maison"] = numbers_in_range[1]
            elif "loyer" in label:
                if "loyer_appartement" not in data_dict:
                    data_dict["loyer_appartement"] = main_value
                    data_dict["loyer_min_appartement"] = numbers_in_range[0]
                    data_dict["loyer_max_appartement"] = numbers_in_range[1]
                else:
                    data_dict["loyer_maison"] = main_value
                    data_dict["loyer_min_maison"] = numbers_in_range[0]
                    data_dict["loyer_max_maison"] = numbers_in_range[1]
        
        #3-Ajouter l'URL pour référence
        data_dict["url"] = url
        data_dict['Code_insee'] = code_insee
        all_data.append(data_dict)
        print(f"Datas ajoutées pour la ville : {ville}")
        
    except Exception as erreur:
        print(f"Erreur pour l'URL {url}: {erreur}")
        error_data.append({
        "url": url,
        "erreur": str(erreur)})
        
    # Pause pour ne pas surcharger le site
    time.sleep(random.uniform(2.3, 3.6))

driver.quit()



Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/basse-goulaine-44115/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/bouguenais-44340/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/les-couets-44340/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/la-chapelle-sur-erdre-44240/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/donges-44480/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/fay-de-bretagne-44130/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/grandchamp-des-fontaines-44119/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/heric-44810/: list index out of range
Erreur pour l'URL https://www.meilleursagents.com/prix-immobilier/basse-indre-44610/: list index out of

In [7]:
df_error_url = pd.DataFrame(error_data)
df_meilleur_agent_PdlL = pd.DataFrame(all_data)

In [9]:
df_meilleur_agent_PdlL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ville                  512 non-null    object 
 1   prix_appartement       512 non-null    float64
 2   min_appartement        512 non-null    float64
 3   max_appartement        512 non-null    float64
 4   prix_maison            512 non-null    float64
 5   min_maison             512 non-null    float64
 6   max_maison             512 non-null    float64
 7   loyer_appartement      512 non-null    float64
 8   loyer_min_appartement  512 non-null    float64
 9   loyer_max_appartement  512 non-null    float64
 10  loyer_maison           497 non-null    float64
 11  loyer_min_maison       497 non-null    float64
 12  loyer_max_maison       497 non-null    float64
 13  url                    512 non-null    object 
 14  Code_insee             512 non-null    object 
dtypes: flo

In [8]:
print(df_meilleur_agent_PdlL.shape)
print(df_error_url.shape)

(512, 15)
(60, 2)


In [25]:
df_duplicate = df_meilleur_agent5.drop_duplicates(subset=['ville'])

In [10]:
df_meilleur_agent_PdlL.to_csv('ma_2.csv',index=False)
df_error_url.to_csv('error_ma_2.csv',index=False)

In [47]:
df_meilleur_agent9['ratio_appt'] = round((df_meilleur_agent9['prix_appartement'] / df_meilleur_agent9['loyer_appartement']),1)

df_meilleur_agent9['ratio_maison'] = round((df_meilleur_agent9['prix_maison'] / df_meilleur_agent9['loyer_maison']),1)

df_meilleur_agent9 = df_meilleur_agent9.sort_values('ratio_appt')